In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# =========================
# 1. PARAMETERS & SETTINGS
# =========================
data_dir = '/mnt/d/RuhunaNew/Academic/Research/Facial_Recog/SampleDataset'  
num_classes = 9
batch_size = 32
num_epochs = 25
learning_rate = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =========================
# 2. DATA TRANSFORMS
# =========================
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# =========================
# 3. DATASET & DATALOADERS
# =========================
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}
train_loader = DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=4)
class_names = image_datasets['train'].classes
print("Classes:", class_names)

# =========================
# 4. MODEL SETUP
# =========================
model = models.densenet161(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

# OPTIONAL: Freeze earlier layers for feature extraction
# for param in model.parameters():
#     param.requires_grad = False
# for param in model.classifier.parameters():
#     param.requires_grad = True

# =========================
# 5. LOSS & OPTIMIZER
# =========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# =========================
# 6. TRAINING & VALIDATION
# =========================
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs} [Train]")
        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            tepoch.set_postfix(loss=loss.item(), acc=100.*correct/total if total > 0 else 0)

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        with tqdm(val_loader, unit="batch") as vtepoch:
            vtepoch.set_description(f"Epoch {epoch+1}/{num_epochs} [Val]")
            for inputs, labels in vtepoch:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()
                vtepoch.set_postfix(loss=loss.item(), acc=100.*val_correct/val_total if val_total > 0 else 0)

    val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100. * val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} -- Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%\n")

# =========================
# 7. SAVE MODEL
# =========================
torch.save(model.state_dict(), 'densenet161_emotion_9class.pth')
print("Training complete. Model saved as 'densenet161_emotion_9class.pth'")


In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model setup (must match your training code)
num_classes = 9
model = models.densenet161(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, num_classes)

# If you used DataParallel, load like this:
state_dict = torch.load('densenet161_emotion_9class.pth', map_location=device)
if 'module.' in list(state_dict.keys())[0]:
    # Remove 'module.' prefix if present
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith('module.') else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(state_dict)

model = model.to(device)
model.eval()


c:\Users\LOQ\anaconda3\envs\pythonenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LOQ\anaconda3\envs\pythonenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [2]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [3]:
# Replace with your actual class names in the order used during training
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise', 'contempt', 'other']


In [ ]:
import cv2
import torch
import numpy as np

# Assume model, preprocess, device, class_names are already defined and loaded

cap = cv2.VideoCapture(0)
emotion = "..."  # Default label

frame_count = 0
predict_every_n = 3  # Predict every 3rd frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % predict_every_n == 0:
        # Optionally, resize frame for faster processing
        small_frame = cv2.resize(frame, (224, 224))
        img = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        img = preprocess(img)
        img = img.unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(img)
            _, pred = torch.max(outputs, 1)
            emotion = class_names[pred.item()]
            print(f"Predicted Emotion: {emotion}")  # Print to console

    # Overlay prediction
    cv2.putText(frame, f'Emotion: {emotion}', (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow('Emotion Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from ultralytics import YOLO

# Load your trained model
model = YOLO("best_emotion_yolov11.pt")

# Class names (must match your training classes EXACTLY)
emotion_classes = ['angry','boring','disgust','fear','happy','neutral','sad','stress','surprise']

def webcam_emotion_detection():
    cap = cv2.VideoCapture(0)
    input_size = 224  # Change to your training size

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Preprocess frame (resize to model input size)
        resized = cv2.resize(frame, (input_size, input_size))
        
        # Run inference on GPU (device=0) or CPU (device='cpu')
        results = model(resized, device=0, verbose=False)[0]  # device=0 for first GPU
        
        # Get prediction
        if results.probs is not None:
            pred_id = results.probs.top1
            confidence = results.probs.top1conf.item()
            label = f"{emotion_classes[pred_id]} ({confidence:.2f})"
            
            # Display prediction
            cv2.putText(frame, label, (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # Print prediction
            print(f"Predicted Emotion: {label}")
        
        cv2.imshow('Emotion Detection', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the detection
webcam_emotion_detection()


: 